In [1]:


from transformers import CLIPModel, CLIPProcessor


/opt/miniconda3/envs/fashion-app/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
CLIPModel.from_pretrained("openai/clip-vit-base-patch32").save_pretrained(
    "clip-model",
    safe_serialization=False
)
CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32").save_pretrained("clip-model")

/opt/miniconda3/envs/fashion-app/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[]

In [10]:
%%bash

torch-model-archiver \
  --model-name clip \
  --version 1.0 \
  --serialized-file clip-model/pytorch_model.bin \
  --handler handler.py \
  --extra-files "clip-model/config.json,clip-model/tokenizer_config.json,clip-model/preprocessor_config.json,clip-model/vocab.json,clip-model/merges.txt,clip-model/special_tokens_map.json" \
  --export-path model-server/model-store 


### Our model saved in the ./model-serer/model-store/clip.mar

In [11]:
%%bash
mkdir clip-model-safe

In [13]:
from transformers import CLIPModel, CLIPProcessor
import torch

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Приводим все параметры к contiguous
for name, param in model.named_parameters():
    param.data = param.data.contiguous()

model.save_pretrained("clip-model-safe", safe_serialization=True)
processor.save_pretrained("clip-model-safe")


[]

In [59]:
%%bash
torch-model-archiver \
  --model-name clip \
  --version 2.0 \
  --serialized-file clip-model-safe/model.safetensors \
  --handler handler.py \
  --extra-files "clip-model-safe/config.json,clip-model-safe/preprocessor_config.json,clip-model-safe/tokenizer_config.json,clip-model-safe/merges.txt,clip-model-safe/vocab.json,clip-model-safe/special_tokens_map.json" \
  --export-path model-server/model-store \
  --force


WARNING - Overwriting model-server/model-store/clip.mar ...


In [48]:
import requests

In [ ]:
requests.options("http://localhost:8081/").json()

In [60]:
requests.get("http://localhost:8081/models").json()

{'models': [{'modelName': 'clip', 'modelUrl': 'clip.mar'}]}

In [61]:
%%bash
curl -X POST "http://localhost:8080/predictions/clip" -F "data=@kitten_small.jpg"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21158  100 13611  100  7547  30546  16937 --:--:-- --:--:-- --:--:-- 47546


{
  "main_style": "Home wear",
  "main_confidence": 12.69,
  "secondary_style": "Formal",
  "secondary_confidence": 12.58,
  "embedding_dim": 512,
  "embedding": [
    0.00821248535066843,
    -0.010961783118546009,
    -0.0017622057348489761,
    -0.01648467220366001,
    -0.013522046618163586,
    -0.03167618438601494,
    -0.020641781389713287,
    0.05358176678419113,
    -0.005341621581465006,
    -0.005364680662751198,
    0.03936677798628807,
    -0.04548771679401398,
    0.08241954445838928,
    -0.006101024802774191,
    0.02829892560839653,
    -0.010074527934193611,
    0.05634405091404915,
    -0.025549300014972687,
    0.014366213232278824,
    0.046248484402894974,
    -0.04974069818854332,
    0.0009133759303949773,
    0.020975187420845032,
    -0.04664948210120201,
    -0.05068955197930336,
    0.012751306407153606,
    0.025020407512784004,
    0.04442749544978142,
    0.03005947731435299,
    0.009091310203075409,
    0.0199413038790226,
    -0.01853013038635254,
   

In [58]:
%%bash
curl http://localhost:8080/predictions/clip -T kitten_small.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7441  100   100  100  7341    252  18504 --:--:-- --:--:-- --:--:-- 18790


{
  "code": 503,
  "type": "InternalServerException",
  "message": "Invalid model predict output"
}
